# Реализация модели двустороннего защищенного чата в мессенджере по спецификации Signal

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# КГПСЧ
from Crypto.Random import get_random_bytes, random

# HKDF
from Crypto.Protocol.KDF import HKDF

# Вспомогательные радости
from collections import namedtuple
import pickle
import binascii

----

In [4]:
def as_hex(blob: bytes):
    """
    Перевод блоба в шестнадцатиричное представление
    """
    
    return binascii.hexlify(blob).decode()

----
## Реализация оберток над криптографическими алгоритмами и примитивами

In [5]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes, associated_data: bytes = b''):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = gost3412.GOST3412Kuznechik(self._key)
        cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, associated_data)
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes, associated_data: bytes = b''):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        try:
            encrypter = gost3412.GOST3412Kuznechik(self._key)
            cipher = mgm.MGM(encrypter.encrypt, KuznyechikMGM._block_size())

            ct_with_tag = ciphertext + tag

            return cipher.open(nonce, ct_with_tag, associated_data)
        except ValueError as e:
            return None
        
        
    @staticmethod
    def _block_size():
        return gost3412.GOST3412Kuznechik.blocksize
    
    
    @staticmethod
    def _generate_nonce():
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

---

In [6]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [7]:
def hash_digest(hasher_type, *args):
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [8]:
class Curve(object):
    """
    Обертка над эллиптической кривой. Для простоты это одна из кривых,
    определенных в ГОСТ 34.10
    """
    
    CURVE = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        
        
    @staticmethod
    def generate_private_key():
        """
        Генерирование закрытого ключа
        """
        
        return gost3410.prv_unmarshal(get_random_bytes(64))
    
    
    @staticmethod
    def get_public_key(private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return gost3410.public_key(Curve.CURVE, private_key)
        
        
    @staticmethod
    def multiply_by_scalar(scalar, point):
        """
        Умножение точки эллиптической кривой на скаляр 
        """

        return Curve.CURVE.exp(scalar, *point)

    
    @staticmethod
    def add_points(lhs, rhs):
        """
        Сложение двух точек эллиптической кривой
        """

        return Curve.CURVE._add(*lhs, *rhs)

----
## Элементы протоколов из спецификации Signal

In [9]:
class KdfChainInternal(object):
    """
    Вспомогательный класс, описывающий KDF-цепочку
    """
    
    def __init__(self, key, key_length):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        self._key_length = key_length
        self._kdf_key = key
    
    
    def reset(self, key):
        """
        Сброс состояния, запись нового ключа в качестве
        первоначального
        """
        
        self._kdf_key = key
        
        
    def _kdf_update(self, data):
        self._kdf_key, message_key = HKDF(self._kdf_key, self._key_length, 
                                          data, Streebog, 2)
        return message_key

In [10]:
class KdfChain(KdfChainInternal):
    """
    Класс, описывающий KDF-цепочку, которая генерирует
    новые ключи только на основе своего состояния
    https://signal.org/docs/specifications/doubleratchet/#kdf-chains
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(KdfChain, self).__init__(key, key_length)
        
        
    def step(self):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(b'\xFF' * 32)

In [11]:
class SymmetricRatchet(KdfChainInternal):
    """
    Симметричный храповик
    Новые ключи вырабатываются на основе выработанного 
    секрета Диффи-Хеллмана
    https://signal.org/docs/specifications/doubleratchet/#symmetric-key-ratchet
    """
    
    def __init__(self, key, key_length=32):
        """
        Инициализация, сохранение начального ключа и 
        длины генерируемых далее ключей
        """
        
        super(SymmetricRatchet, self).__init__(key, key_length)
        
        
    def step(self, serialized_dh_key):
        """
        Шаг цепочки, выработка нового ключа для KDF и
        возвращаемого ключа
        """
        
        return self._kdf_update(serialized_dh_key)

-----

In [12]:
class DiffieHellmanRatchet(object):
    """
    DH-храповик
    https://signal.org/docs/specifications/doubleratchet/#diffie-hellman-ratchet
    """
    
    PRIVATE = 0
    PUBLIC  = 1
    
    
    def __init__(self):
        """
        Инициализация, выработка начальной ключевой пары
        """
        
        self._current_foreign_public_key = None
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
    
    def is_uninitialized(self):
        """
        Проверка того, что открытый ключ другой стороны отсутствует
        """
        
        return self._current_foreign_public_key is None
    
    
    def initialize(self, public_key):
        """
        Инициализация открытого ключа другой стороны
        """
        
        self._current_foreign_public_key = public_key
        
        
    def is_same_public_key(self, public_key):
        """
        Проверка равенства открытых ключей
        """
        
        if self._current_foreign_public_key is None:
            return False
        
        return public_key == self._current_foreign_public_key
    
    
    def step(self, public_key=None):
        """
        Выработка общего секрета Диффи-Хеллмана
        """
        
        if self._current_key_pair is None:
            raise RuntimeError('Uninitialized DH Ratchet used to produce a shared secret')
        
        if public_key is not None:
            self._current_foreign_public_key = public_key
            
        return Curve.multiply_by_scalar(self._current_key_pair[DiffieHellmanRatchet.PRIVATE], 
                                        self._current_foreign_public_key)
    
    
    def generate_new_key_pair(self):
        """
        Генерирование новой ключевой пары
        """
        
        self._current_key_pair = DiffieHellmanRatchet._generate_key_pair()
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._current_key_pair[DiffieHellmanRatchet.PUBLIC]
    
    
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return private_key, public_key

----

In [13]:
#
# Описание сути следует смотреть здесь:
# https://signal.org/docs/specifications/x3dh/#publishing-keys
#

KeyBundle = namedtuple('KeyBundle', ['IK',    # Ключ-идентификатор (Identity key)
                                     'SPK',   # Подписанный предключ (Signed prekey)
                                     'Sig',   # Подпись предключа (Prekey signature)
                                     'OPKs',  # Коллекция одноразовых предключей (One-time prekeys)
                                    ])

In [14]:
Message = namedtuple('Message', ['public_key',  # Открытый ключ DH отправляющего
                                 'id',          # Идентитфикатор сообщения
                                 'data',        # Данные сообщения
                                ])

----
## Участники протоколов из спецификации Signal

In [15]:
class SignalServer(object):
    """
    Класс сервера Signal
    """
    
    def __init__(self):
        """
        Инициализация
        """
        
        #
        # Коллекция ключевых данных клиентов, индексируемая по номеру телефона
        #
        
        self._key_bundles = {}
        
        #
        # Коллекция очередей сообщения, индексируемая по паре номеров, 
        # представляющих направление передачи
        #
        
        self._messages = {}
        
        
    def register(self, phone_number: str, keys: KeyBundle):
        """
        Регистрация пользователя по номеру телефона. Сохраняет набор ключей
        """
        
        if phone_number in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is already registered')
            
        trace('[SignalServer]', f'Registered client with phone number {phone_number}')
            
        self._key_bundles[phone_number] = keys
        
        
    def get_public_keys(self, phone_number: str):
        """
        Получение ключей пользователя по его номеру телефона
        """
        
        if phone_number not in self._key_bundles:
            raise ValueError(f'Client with phone number {phone_number} is not registered yet')
            
        #
        # Достану ключи
        #
        
        keys = self._key_bundles[phone_number]
        one_time_keys = keys.OPKs
        
        #
        # Возьму случайный из одноразовых ключей
        #
        
        idx = random.randint(0, len(one_time_keys) - 1)
        result = KeyBundle(keys.IK, keys.SPK, keys.Sig, (one_time_keys.pop(idx), idx))
        
        trace('[SignalServer]', f'Using OPK at {idx}')
        
        #
        # Сохраню обновленный пакет ключей (без одного одноразового)
        #
        
        self._key_bundles[phone_number] = KeyBundle(keys.IK, keys.SPK, keys.Sig, one_time_keys)
        
        return result
    
    
    def push_message(self, phone_from: str, phone_to: str, message: Message):
        """
        Помещение сообщения в очередь
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            self._messages[search_key] = []
            
        trace('[SignalServer]', f'Saved message with id {message.id} from {phone_from} to {phone_to}')
            
        self._messages[search_key].append(message)
        
        
    def peek_message(self, phone_from, phone_to):
        """
        Получение сообщения из очереди
        """
        
        search_key = (phone_from, phone_to)
        
        if search_key not in self._messages:
            return None
        
        if self._messages[search_key] == []:
            return None
        
        return self._messages[search_key].pop(0)

In [16]:
class SignalClient(object):
    """
    Класс клиента Signal
    """
    
    PRIVATE = 0
    PUBLIC  = 1
    
    X3DH_MSG_ID      = 'X3DH_MSG'
    DH_RATCHET_PK_ID = 'DH_RATCHET_PK'
    
    RECEIVED_FLAG     = 0
    DH_RATCHET        = 1
    ROOT_KDF_CHAIN    = 2
    SENDING_RATCHET   = 3
    RECEIVING_RATCHET = 4
    MSG_ID            = 5
    
    
    def __init__(self, phone_number, one_time_keys=10, *, fake_ik=False):
        """
        Инициализация, генерирование ключей
        """
        
        self._phone_number  = phone_number
        
        self._identity_key  = SignalClient._generate_key_pair()
        self._signed_prekey = SignalClient._generate_key_pair()
        self._ot_prekeys    = [SignalClient._generate_key_pair() for _ in range(one_time_keys)]
        
        digest = hash_digest(Streebog, pickle.dumps(self._signed_prekey[SignalClient.PUBLIC]))
        self._signature = gost3410.sign(Curve.CURVE, 
                                        self._identity_key[SignalClient.PRIVATE], 
                                        digest)
        
        if fake_ik:
            #
            # Демонстрация неудачного выполнения, заменю идентификационный ключ
            #
            
            self._identity_key  = SignalClient._generate_key_pair()
        
        self._sessions = {}
        
        trace('[SignalClient]', f'Identity key: {self._identity_key}')
        trace('[SignalClient]', f'Signed prekey: {self._signed_prekey}')
        trace('[SignalClient]', f'Signature: {as_hex(self._signature)}')
        trace('[SignalClient]', f'One time keys: {len(self._ot_prekeys)}')
        
        
    def send_message(self, phone_number: str, message: str, server: SignalServer):
        """
        Отправка сообщения, установление сессии, если нужно
        """
        
        if phone_number not in self._sessions:
            #
            # Сессии еще нет, поэтому установлю соединение
            #
            
            return self._x3dh_init_session(phone_number, message, server)
        
        #
        # Так как протокол X3DH асинхронный, то я прямо сразу и отправлю сообщение
        #
        
        if self._sessions[phone_number][SignalClient.RECEIVED_FLAG]:
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            self._sessions[phone_number][SignalClient.RECEIVED_FLAG] = False
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key   = self._sessions[phone_number][SignalClient.DH_RATCHET].step()
            
            trace('[SignalClient]', 'Root KDF chain step')
            send_key = self._sessions[phone_number][SignalClient.ROOT_KDF_CHAIN].step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Sending ratchet reset')
            self._sessions[phone_number][SignalClient.SENDING_RATCHET].reset(send_key)
            
        #
        # Вырабатывается ключ с помощью симметричного храповика для отправки
        # На ключе шифруется само сообщение
        #
        
        trace('[SignalClient]', 'Sending ratchet step')
        message_key = self._sessions[phone_number][SignalClient.SENDING_RATCHET].step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as encryption key')
        encrypted_message = KuznyechikMGM(message_key).encrypt(message.encode())
        
        message_id = self._sessions[phone_number][SignalClient.MSG_ID]
        public_key = self._sessions[phone_number][SignalClient.DH_RATCHET].public_key
        
        self._sessions[phone_number][SignalClient.MSG_ID] += 1
        
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     message_id, 
                                                                     encrypted_message))
        
        
    def receive_message(self, phone_number: str, server: SignalServer):
        """
        Прием сообщения, установление сессии, если надо
        """
        
        message = server.peek_message(phone_number, self.phone_number)
        if message is None:
            trace('[SignalClient]', f'No message for {self.phone_number} from {phone_number}')
            return
        
        public_key, message_id, data = message
        
        trace('[SignalClient]', f'Received message with id {message_id} from {phone_number}')
        
        if message_id == SignalClient.X3DH_MSG_ID:
            #
            # Это инициирующее сообщения для установки сессии
            #
            
            return self._x3dh_accept_session(phone_number, data, server)
        
        if message_id == SignalClient.DH_RATCHET_PK_ID:
            #
            # Это нам ключ открытый прислали для инициализации
            #
            
            if not self._sessions[phone_number][SignalClient.DH_RATCHET].is_uninitialized():
                raise RuntimeError(f'Cannot initialize DH ratchet for {self.phone_number} twice')
                
            return self._sessions[phone_number][SignalClient.DH_RATCHET].initialize(public_key)
        
        #
        # Во-первых, факт приема зафиксирую 
        #
        
        self._sessions[phone_number][SignalClient.RECEIVED_FLAG] = True
        
        #
        # Не сменился ли открытый ключ собеседника?
        #
        
        if not self._sessions[phone_number][SignalClient.DH_RATCHET].is_same_public_key(public_key):
            #
            # Требуется шаг DH-храповика, так как сменилось направление передачи
            #
            
            trace('[SignalClient]', 'DH ratchet step')
            dh_key = self._sessions[phone_number][SignalClient.DH_RATCHET].step(public_key)
            self._sessions[phone_number][SignalClient.DH_RATCHET].generate_new_key_pair()
            
            trace('[SignalClient]', 'Root KDF chain step')
            recv_key = self._sessions[phone_number][SignalClient.ROOT_KDF_CHAIN].step(pickle.dumps(dh_key))
            
            trace('[SignalClient]', 'Receiving ratchet reset')
            self._sessions[phone_number][SignalClient.RECEIVING_RATCHET].reset(recv_key)
            
        #
        # Делаю шаг храповика приема
        #
            
        trace('[SignalClient]', 'Receiving ratchet step')
        message_key = self._sessions[phone_number][SignalClient.RECEIVING_RATCHET].step()
        
        trace('[SignalClient]', f'Use {as_hex(message_key)} as decryption key')
        decrypted_message = KuznyechikMGM(message_key).decrypt(*data)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
            
        return decrypted_message.decode()
        
        
    @property
    def phone_number(self):
        """
        Получение номер телефона
        """
        
        return self._phone_number
    
    
    @property
    def public_keys(self):
        """
        Получение открытых ключей
        """
        
        return KeyBundle(self._identity_key[SignalClient.PUBLIC], 
                         self._signed_prekey[SignalClient.PUBLIC], 
                         self._signature, 
                         [prekey[SignalClient.PUBLIC] for prekey in self._ot_prekeys])
        
        
    def _x3dh_init_session(self, phone_number: str, message: str, server: SignalServer):
        IKb, SPKb, Sigb, (OPKb, idx) = server.get_public_keys(phone_number)
        
        #
        # Проверяю подпись
        #
        
        digest = hash_digest(Streebog, pickle.dumps(SPKb))
        if not gost3410.verify(Curve.CURVE, IKb, digest, Sigb):
            trace('[SignalClient]', f'Cannot verify signature of client with phone {phone_number}')
            return
        
        #
        # Эфемерный ключ
        #
        
        EKa = SignalClient._generate_key_pair()
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._identity_key, SPKb)
        DH2 = SignalClient._dh(EKa, IKb)
        DH3 = SignalClient._dh(EKa, SPKb)
        DH4 = SignalClient._dh(EKa, OPKb)
        
        #
        # Вырабатываю ключ и создаю сообщение
        # В качестве данных я отправлю шифртекст сообщения,
        # идентификационный ключ, эфемерный ключ и индекс
        # использованного одноразового ключа
        #
        # Протокол разрешает использовать выработанный ключ прямо
        # для зашифрования первого сообщения: An initial ciphertext 
        # encrypted with some AEAD encryption scheme [4] using AD 
        # as associated data and using an encryption key which is 
        # either SK or the output from some cryptographic PRF keyed by SK.
        #
        
        AD = pickle.dumps(self._identity_key[SignalClient.PUBLIC]) + pickle.dumps(IKb)
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        #
        # А теперь требуется создать состояние
        #
        
        self._initialize_state(phone_number, root_key)
        
        encrypted_message = KuznyechikMGM(root_key).encrypt(message.encode(), AD)
        message = (encrypted_message, self._identity_key[SignalClient.PUBLIC], 
                   EKa[SignalClient.PUBLIC], idx)
        
        server.push_message(self.phone_number, 
                            phone_number, 
                            Message(None, SignalClient.X3DH_MSG_ID, message))
        
    
    def _x3dh_accept_session(self, phone_number: str, x3dh_parameters: Message, server: SignalServer):
        encrypred_message, IKa, EKa, idx = x3dh_parameters
        
        #
        # Получу нужный одноразовый ключ
        #
        
        OPKb = self._ot_prekeys.pop(idx)
        
        #
        # Выработка общих секретов
        #
        
        DH1 = SignalClient._dh(self._signed_prekey, IKa)
        DH2 = SignalClient._dh(self._identity_key, EKa)
        DH3 = SignalClient._dh(self._signed_prekey, EKa)
        DH4 = SignalClient._dh(OPKb, EKa)
        
        #
        # Вырабатываю ключ и читаю сообщение
        #
        
        AD = pickle.dumps(IKa) + pickle.dumps(self._identity_key[SignalClient.PUBLIC])
        trace('[SignalClient]', f'Use {as_hex(AD)} as associated data')
        
        root_key = SignalClient._produce_root_key(DH1, DH2, DH3, DH4)
        trace('[SignalClient]', f'Use {as_hex(root_key)} as root key')
        
        decrypted_message = KuznyechikMGM(root_key).decrypt(*encrypred_message, AD)
        
        if decrypted_message is None:
            trace('[SignalClient]', 'Cannot decrypt message')
            return
        
        #
        # Теперь нужно инициализировать состояние и отослать открытый ключ
        #
        
        self._initialize_state(phone_number, root_key)
        
        public_key = self._sessions[phone_number][SignalClient.DH_RATCHET].public_key
        server.push_message(self.phone_number, phone_number, Message(public_key, 
                                                                     SignalClient.DH_RATCHET_PK_ID, 
                                                                     None))
        
        return decrypted_message.decode()
        
        
    def _initialize_state(self, phone_number: str, root_key):
        self._sessions[phone_number] = [True, 
                                        DiffieHellmanRatchet(), 
                                        SymmetricRatchet(root_key), 
                                        KdfChain(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                        KdfChain(b'\x00', KuznyechikMGM.KEY_SIZE), 
                                        0,
                                       ]
        
        
    def _try_mimic(self, phone_number, session_phone_number, session):
        self._phone_number = phone_number
        self._sessions[session_phone_number] = session.copy()
        self._sessions[session_phone_number][SignalClient.DH_RATCHET].generate_new_key_pair()
        
        trace('[SignalClient]', f'User now tries to be a user with phone {phone_number}')
        
        
    @staticmethod
    def _generate_key_pair():
        private_key = Curve.generate_private_key()
        public_key  = Curve.get_public_key(private_key)
        
        return private_key, public_key
    
    
    @staticmethod
    def _dh(key_pair, public_key):
        return Curve.multiply_by_scalar(key_pair[SignalClient.PRIVATE], public_key)
    
    
    @staticmethod
    def _produce_root_key(DH1, DH2, DH3, DH4):
        kdf_data = pickle.dumps(DH1) + pickle.dumps(DH2) + pickle.dumps(DH3) + pickle.dumps(DH4)
        return HKDF(kdf_data, KuznyechikMGM.KEY_SIZE, b'\x00' * 16, Streebog, 1)

----
## Демонстрация удачного выполнения протоколов

In [17]:
# Создаю сервер
srv = SignalServer()

In [18]:
# Пользователь Алиса
alice = SignalClient('+7 921 059-87-72')

[TRACE] [SignalClient] Identity key: (9321895140060955662932515934312588729541099390398997757103464850837339686392582139493483424184244332807659658703342726309294648688466419615426931245922774, (12854680721013462168309964974609070626264448140731148777676220526717884133992962274423607754565060110393486450769704634110937844329959551816071001709047837, 8635931216381818097982374394604867527031056130616162809798738899066975378894666012997537122715077428970721104526241252171417093234333129922022588221334509))
[TRACE] [SignalClient] Signed prekey: (1893046315884572570379007513686618323990883029244683296250113039301124245394750437797517151202083285211133481424525198718846311983842468283284692627558365, (12904367137892249108264512100572607340944636015926195503348882735912340629351554898325047911964227240531119973393596993709562984235589672797734138203138296, 890597397297768792207020446683865428702113614691195390392760126524204563325007552039382792015319052113958462507141996596470747221136790111

In [19]:
# Пользователь Боб
bob = SignalClient('+7 991 174-51-34')

[TRACE] [SignalClient] Identity key: (8122642917616793878861762261169651355521853443934668968082589482383034540768852016952639647700083905831393246495311552808065564268931648804713700294777642, (6860537459622850679068513744364983228766481233296418525123447074445512153787794745691503378241723033436931667111209898223551924780875009778784713422356737, 2967288941038091552188548782917287362806747390484090710921132068678908170536298764667837230096022539930187592447831521803995323327920099778237988386767197))
[TRACE] [SignalClient] Signed prekey: (13063823150231078463006244866989365900754980721668182425758784917756889339037510235300727203557933058384665370636809278283838761673578199296566773942167902, (4274157283205051331185589665221499990043631566491882383427985176040861603824545067900613802806543836269484682610465091135249435176970329318812926662913928, 6669072787069920201955935233220837223047087501363384478209288600191910125602066385693121924433709484747228999947728747686560627288496679156

In [20]:
# Регистрирую всех 
srv.register(alice.phone_number, alice.public_keys)
srv.register(bob.phone_number, bob.public_keys)

[TRACE] [SignalServer] Registered client with phone number +7 921 059-87-72
[TRACE] [SignalServer] Registered client with phone number +7 991 174-51-34


In [21]:
# Алиса пытается пообщаться с Бобом
alice.send_message(bob.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 2
[TRACE] [SignalClient] Use 80049589000000000000008a411d70f9e2ab7d2128c4da6719a53bde000aa0f1b7f732fdab3d7a8a8c93b46dba85c43c2eb942837eb31b2a2086461cbbc48687fa6aa7c4ef224c957bda5e70f5008a41eda3554f68857f8258c3cd10c2cd4ca7d829d801c19ca86eca039148af28b53901d168686d55a1d1f113bd9d9d3636297e8fccd347ecd782cb21c9ea6f8de3a40086942e80049588000000000000008a4101416f93941d3fe59e534540d94a3f87bc6db51ee3650784bb92a797def78b71f9265d810fab19affb24b327326cb09444621d17397e04d38bbf297ccb9bfd82008a405d012eaa2b751bf8d57570cdbdf8b036e0ce7e00532f44e61995a4d17d9b40a14d94588249268f49e447b26d07de8738424876587f563ec48e82536164cea73886942e as associated data
[TRACE] [SignalClient] Use a0e876a03849d0f6762d82e0f69b32a7ee9cfec7b6a81c7106e878b2a93c186a as root key
[TRACE] [SignalServer] Saved message with id X3DH_MSG from +7 921 059-87-72 to +7 991 174-51-34


In [22]:
# Боб полчает сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049589000000000000008a411d70f9e2ab7d2128c4da6719a53bde000aa0f1b7f732fdab3d7a8a8c93b46dba85c43c2eb942837eb31b2a2086461cbbc48687fa6aa7c4ef224c957bda5e70f5008a41eda3554f68857f8258c3cd10c2cd4ca7d829d801c19ca86eca039148af28b53901d168686d55a1d1f113bd9d9d3636297e8fccd347ecd782cb21c9ea6f8de3a40086942e80049588000000000000008a4101416f93941d3fe59e534540d94a3f87bc6db51ee3650784bb92a797def78b71f9265d810fab19affb24b327326cb09444621d17397e04d38bbf297ccb9bfd82008a405d012eaa2b751bf8d57570cdbdf8b036e0ce7e00532f44e61995a4d17d9b40a14d94588249268f49e447b26d07de8738424876587f563ec48e82536164cea73886942e as associated data
[TRACE] [SignalClient] Use a0e876a03849d0f6762d82e0f69b32a7ee9cfec7b6a81c7106e878b2a93c186a as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 991 174-51-34 to +7 921 059-87-72


'Hi!'

In [23]:
# Это небольшой костыль, тут я получаю инициализирующее 
# сообщение с открытым ключом Боба
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id DH_RATCHET_PK from +7 991 174-51-34


In [24]:
# Алиса отсылает сообщение
alice.send_message(bob.phone_number, "What's up?", srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use cea971417001fa5f08f1aa4f61b997dab18f422e4826a56d66847542aacbbaee as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 921 059-87-72 to +7 991 174-51-34


In [25]:
# Боб принимает и расшифровывает
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use cea971417001fa5f08f1aa4f61b997dab18f422e4826a56d66847542aacbbaee as decryption key


"What's up?"

In [26]:
# Можно послать несколько сообщений подряд
bob.send_message(alice.phone_number, 'Some dummy phrase just to continue the dialog', srv)
bob.send_message(alice.phone_number, 'And another one', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 7a864a802084ce17b6d5b41d5f4be6b5939da48cd210d7b057e683fc35850b84 as encryption key
[TRACE] [SignalServer] Saved message with id 0 from +7 991 174-51-34 to +7 921 059-87-72
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 6fb08539bd56db0fb81f99d820aebbd7e4885214e1a15752142afd14b6aefe66 as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 991 174-51-34 to +7 921 059-87-72


In [27]:
# И получить оба. Одно...
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 0 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 7a864a802084ce17b6d5b41d5f4be6b5939da48cd210d7b057e683fc35850b84 as decryption key


'Some dummy phrase just to continue the dialog'

In [28]:
# ... за другим
alice.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 991 174-51-34
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 6fb08539bd56db0fb81f99d820aebbd7e4885214e1a15752142afd14b6aefe66 as decryption key


'And another one'

In [29]:
# Можно снова сменить направление отправки и 
# посмотреть, что произошел шаг DH-храповика
alice.send_message(bob.phone_number, 'Yet another DH ratchet step demo', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use 62ec55f2ff41ae993933a890019b88e6d737be0c223cbfc6dfdad6d75c056ab3 as encryption key
[TRACE] [SignalServer] Saved message with id 1 from +7 921 059-87-72 to +7 991 174-51-34


In [30]:
# А на другой строне надо получить сообщение
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 1 from +7 921 059-87-72
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use 62ec55f2ff41ae993933a890019b88e6d737be0c223cbfc6dfdad6d75c056ab3 as decryption key


'Yet another DH ratchet step demo'

In [31]:
# Можно третьего пользователя создать
carol = SignalClient('+7 970 505-44-25')
srv.register(carol.phone_number, carol.public_keys)

[TRACE] [SignalClient] Identity key: (11926474360601648416693343534955120095013027220782406209551480392024326297870099215573794397373011966340932732770997562007866963562836353929512698401387182, (11910105930728788711855569255348059804433474310341871865874467506443053963274566637379166439688510083299817291192159418067078457918579845198907296458510548, 13037040522275365794996265149533417981265926570641663885710469986289923851816881951960074418556325808196765383225884130508673564009151018003983462897334700))
[TRACE] [SignalClient] Signed prekey: (4976240640723849603371332674682049079482419203633411558286177722017153943596727201327318674975144296994519041644585376722598954589096428801248501434506187, (2075282459460970740728758642595701646487318516566884459654033724887370847935519555630928559157587075828514002928808943782514532396104199721126648819048040, 11280681221236111493066928250499953128964937954984414673643937053562486645509057863357999551405422316706375870871916555892283379309337522

In [32]:
alice.send_message(bob.phone_number, "I'm making new friends", srv)
alice.send_message(carol.phone_number, 'Hi, Carol!', srv)

[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use cda99692824b6612601cd7a305d3402780d5e3f76cca4147ee2e278e38b8133f as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 921 059-87-72 to +7 991 174-51-34
[TRACE] [SignalServer] Using OPK at 3
[TRACE] [SignalClient] Use 80049589000000000000008a411d70f9e2ab7d2128c4da6719a53bde000aa0f1b7f732fdab3d7a8a8c93b46dba85c43c2eb942837eb31b2a2086461cbbc48687fa6aa7c4ef224c957bda5e70f5008a41eda3554f68857f8258c3cd10c2cd4ca7d829d801c19ca86eca039148af28b53901d168686d55a1d1f113bd9d9d3636297e8fccd347ecd782cb21c9ea6f8de3a40086942e80049589000000000000008a41d49098e31e2aff6587d49197d2d8f40ad8029b0b34adc8ac36e08c003dc22b6b9310766d0dc31c6e34498bd3d862de4ead2a9a67ace801c01c8b0152976267e3008a41acb960599c920717e54ac83e6a422dc3a09601c7b996727c9400e4ed68b4b63e01307e163f55fa0104f0a4a19cf2be0cbfd22732dbe0db082620d08906baebf80086942e as associated data
[TRACE] [SignalClient] Use 13931da6944ffe785cff1b9d4fabbb4579d16ce4d9292b42afb31

In [33]:
carol.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id X3DH_MSG from +7 921 059-87-72
[TRACE] [SignalClient] Use 80049589000000000000008a411d70f9e2ab7d2128c4da6719a53bde000aa0f1b7f732fdab3d7a8a8c93b46dba85c43c2eb942837eb31b2a2086461cbbc48687fa6aa7c4ef224c957bda5e70f5008a41eda3554f68857f8258c3cd10c2cd4ca7d829d801c19ca86eca039148af28b53901d168686d55a1d1f113bd9d9d3636297e8fccd347ecd782cb21c9ea6f8de3a40086942e80049589000000000000008a41d49098e31e2aff6587d49197d2d8f40ad8029b0b34adc8ac36e08c003dc22b6b9310766d0dc31c6e34498bd3d862de4ead2a9a67ace801c01c8b0152976267e3008a41acb960599c920717e54ac83e6a422dc3a09601c7b996727c9400e4ed68b4b63e01307e163f55fa0104f0a4a19cf2be0cbfd22732dbe0db082620d08906baebf80086942e as associated data
[TRACE] [SignalClient] Use 13931da6944ffe785cff1b9d4fabbb4579d16ce4d9292b42afb31a509ca9295f as root key
[TRACE] [SignalServer] Saved message with id DH_RATCHET_PK from +7 970 505-44-25 to +7 921 059-87-72


'Hi, Carol!'

In [34]:
bob.receive_message(alice.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 921 059-87-72
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use cda99692824b6612601cd7a305d3402780d5e3f76cca4147ee2e278e38b8133f as decryption key


"I'm making new friends"

----
## Демонстрация неуспешного выполнения протоколов

In [35]:
# Пользователь, который как-то украл подпись и подписанный ключ
# Но его идентификационный ключ не позволяет проверить подпись
mallory = SignalClient('+1 202-918-2132', fake_ik=True)

[TRACE] [SignalClient] Identity key: (2003199404045945390096057511327513325650552243236762955149555609776212935325169800359706736938338393559588386436559520159469280213695548019705439780656854, (7461228693502033414410521629393918672279710896132839414067431062699310759537179706997563582061498472405773814786942497144861773770228842579880375149051929, 11268950046914826575741721826948180290814083033979953867798744791343154249694909215693828054335968774960626432660512219649296894234824615318541385914093361))
[TRACE] [SignalClient] Signed prekey: (12936388877466676890285493541773678421927278603707437811703199146374766669604865421793556583960973776123685752770097550399105115963277612196100456974059681, (2523104982963365956113993258476167079250550135767524802870177872009525916796243054022587244916927029199123916550400933275995148815328377887049008549232060, 448528796179718975291649839754351628793847306886570795789315195112658546089820279932048364186508107566821978074503115343735086421656510829

In [36]:
# Надо зарегистрироваться
srv.register(mallory.phone_number, mallory.public_keys)

[TRACE] [SignalServer] Registered client with phone number +1 202-918-2132


In [37]:
# Алиса получает ключи с сервера и обнаруживает несоответствие
alice.send_message(mallory.phone_number, 'Hi!', srv)

[TRACE] [SignalServer] Using OPK at 6
[TRACE] [SignalClient] Cannot verify signature of client with phone +1 202-918-2132


In [38]:
# Еще один пользователь-враг -- Ева
eve = SignalClient('+1 539-567-2473')

[TRACE] [SignalClient] Identity key: (10764332129341309203792665542429430781232766768419358748795590904130936970193433040167974066976016885242391408309347574161570320308011729105830445715364227, (4918933871028431068994146496745285663893231998660578313074048444751335622108422859486898041343846710326431357405075063831201268065333447618883465765072964, 9654946095969164342041676471352749355111294790413116032844782511923440530677833517090345960698922294124806184384520591474837526081198671887316389516840939))
[TRACE] [SignalClient] Signed prekey: (6739156483557535439140419684440262265120237418504656969859573901390526367428137325598886240708766535550065386300030793178215194447542918686761849997032901, (1398873284595937787071606584622826196970637446839426281575678589497532781880367764813345760157663217022117588842533240634576183810725308865140686333255439, 1225204779693063843465242640572068532101856758812400336987563801142872782670845237420589303372354845460784370885212351347136217484589938703

In [39]:
# Она пытается прикинуться Алисой в разговоре с Бобом
eve._try_mimic(alice.phone_number, bob.phone_number, 
               alice._sessions[bob.phone_number])

[TRACE] [SignalClient] User now tries to be a user with phone +7 921 059-87-72


In [40]:
# Боб отправляет Алисе соообщение
bob.send_message(alice.phone_number, 'Hi!', srv)

[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Sending ratchet reset
[TRACE] [SignalClient] Sending ratchet step
[TRACE] [SignalClient] Use acdd53c20152179681d0b49a14430cb3713f4e44c87bcd86b977053b25de0e33 as encryption key
[TRACE] [SignalServer] Saved message with id 2 from +7 991 174-51-34 to +7 921 059-87-72


In [41]:
# А вот получить его пытается Ева, но расшифровать не получится
eve.receive_message(bob.phone_number, srv)

[TRACE] [SignalClient] Received message with id 2 from +7 991 174-51-34
[TRACE] [SignalClient] DH ratchet step
[TRACE] [SignalClient] Root KDF chain step
[TRACE] [SignalClient] Receiving ratchet reset
[TRACE] [SignalClient] Receiving ratchet step
[TRACE] [SignalClient] Use ec3149ec47178b6e5b28ba7437d68f2a564331af0133ebcef63129755a4fd510 as decryption key
[TRACE] [SignalClient] Cannot decrypt message
